# 2m99

In [1]:
using Printf, DelimitedFiles
using Plots
using DataFrames
using Revise, MDToolbox

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1423
┌ Info: Precompiling MDToolbox [7965e2f0-a35b-11e8-0207-19c030dd2def]
└ @ Base loading.jl:1423


In [2]:
function read_ascii(fname)
    lines = readlines(fname)
    natom = length(lines) - 1
    xyz = zeros(Float64, 1, natom*3)
    for i = 1:natom
        xyz[1, 3*(i-1)+1] = parse(Float64, lines[i+1][31:38])
        xyz[1, 3*(i-1)+2] = parse(Float64, lines[i+1][39:46])
        xyz[1, 3*(i-1)+3] = parse(Float64, lines[i+1][47:54])
    end
    TrjArray(xyz=xyz)
end

read_ascii (generic function with 1 method)

In [3]:
fnames = []
omega = []
phi = []
prou = []
d = []
score = []
rmsd = []
struc = []

Any[]

In [4]:
nrun = 1000

1000

In [5]:
for irun = 1:nrun

    for i in 1:100
        filename = "$(irun)/complex.$(i).pdb"
        #t = mdload(filename)
        #t = MDToolbox.readpdb_biostructures(filename)
        t = read_ascii(filename)
        t1 = t[:, 1:446]
        t2 = t[:, 447:end]
        r = MDToolbox.compute_skrew(t1, t2)
        push!(fnames, filename)
        push!(omega, r.omega)
        push!(phi, r.phi / (2.0 * pi) * 360.0)
        push!(prou, r.prou)
        push!(d, r.d)
        push!(struc, t)
        #t_ca = t["atomname CA"]
        #push!(rmsd, compute_rmsd_permute(ref, t_ca, 2))
    end

    s = readdlm("$(irun)/zdock.out", skipstart=5)
    score = [score; s[1:100, end]]
end

In [6]:
df = DataFrame(fnames=fnames, score=score, omega=omega, phi=phi, prou=prou, d=d, struc=struc)

,fnames,score,omega,phi,prou
,Any,Any,Any,Any,Any
1,1/complex.1.pdb,1646.24,"[-0.805249, 0.582729, 0.109552]",167.029,"[-2.86018, -2.90684, -5.56133]"
2,1/complex.2.pdb,1644.4,"[0.0371922, -0.995517, 0.0869669]",176.439,"[-9.41837, -0.757952, -4.64847]"
3,1/complex.3.pdb,1585.86,"[0.968191, 0.11526, -0.222082]",175.092,"[-1.51944, 0.0777383, -6.5838]"
4,1/complex.4.pdb,1577.66,"[0.290913, 0.910902, -0.29262]",174.078,"[-7.69357, 0.746557, -5.32471]"
5,1/complex.5.pdb,1533.09,"[0.83827, -0.53273, -0.116194]",163.435,"[-2.18853, -2.10672, -6.13001]"
6,1/complex.6.pdb,1519.82,"[-0.272574, -0.935829, 0.223446]",170.907,"[-7.75707, 0.941257, -5.52043]"
7,1/complex.7.pdb,1514.35,"[0.856241, -0.513057, -0.060194]",176.651,"[-1.77223, -2.27504, -5.81839]"
8,1/complex.8.pdb,1511.18,"[0.783555, -0.621293, 0.00604816]",168.367,"[-2.62678, -3.36159, -5.01081]"
9,1/complex.9.pdb,1504.29,"[-0.114989, -0.987773, 0.105273]",179.183,"[-7.36353, 0.260465, -5.59919]"


In [7]:
df_org = deepcopy(df);

In [8]:
#using BSON: @save, @load
#@save "skrew_parameters.bson" fnames score omega phi prou d df rmsd

In [9]:
#using BSON: @save, @load
#@save "df.bson" df

In [10]:
using JLD2

save("skrew_parameters.jld2", "fnames", fnames, "score", score, "omega", omega, "phi", phi, "prou", prou, "d", d, "df", df, "rmsd", rmsd)
save("df.jld2", "df", df)